In [23]:
from sklearn.neighbors import BallTree
import numpy as np
import pandas as pd
from math import radians
from io import StringIO



In [13]:
# https://stackoverflow.com/questions/56862277/interpreting-sklearn-haversine-outputs-to-kilometers

earth_radius = 6371000 # meters in earth
test_radius = 1300000 # meters

test_points = [[32.027240,-81.093190],[41.981876,-87.969982]]
test_points_rad = np.array([[radians(x[0]), radians(x[1])] for x in test_points ])

tree = BallTree(test_points_rad, metric = 'haversine')
ind,results = tree.query_radius(test_points_rad, r=test_radius/earth_radius) #return_distance  = True
# print(ind)
print(results * earth_radius/1000)

[   0. 6371.]


In [35]:
# https://stackoverflow.com/questions/61952561/how-do-i-find-the-neighbors-of-points-containing-coordinates-in-python

cities = [
    ['B', 50.94029883, 7.019146728], 
    ['C', 50.92073002, 6.975268711], 
    ['D', 50.99807758, 6.980865543],
    ['E', 50.98074288, 7.035060206],
    ['F', 51.00696972, 7.035993783],
    ['G', 50.97369889, 6.928538763],
    ['H', 50.94133859, 6.927878587],
    ['A', 50.96712502, 6.977825322]
]

cities_np = np.array(cities)

df = pd.DataFrame(cities_np, columns=['City', 'Latitude', 'Longitude'])


# print(df[['Latitude', 'Longitude']].values.astype(float))

# print(df)

tree = BallTree(np.deg2rad(df[['Latitude', 'Longitude']].values.astype(float)), metric='haversine')

# Setup distance queries (points for which we want to find nearest neighbors)
other_data = """NAME Latitude Longitude
B_alt 50.94029883 7.019146728
C_alt 50.92073002 6.975268711"""

df_other = pd.read_csv(StringIO(other_data), sep = ' ')

query_lats = df_other['Latitude']
query_lons = df_other['Longitude']

distances, indices = tree.query(np.deg2rad(np.c_[query_lats, query_lons]), k = 3)

r_km = 6371 # multiplier to convert to km (from unit distance)


for name, d, ind in zip(df_other['NAME'], distances, indices):
    print(f"NAME {name} closest matches:")
    print(name, d, ind)
    for i, index in enumerate(ind):
        print(f"\t{df['City'][index]} with distance {d[i]*r_km:.4f} km")


NAME B_alt closest matches:
B_alt [0.         0.00059128 0.00065239] [0 1 7]
	B with distance 0.0000 km
	C with distance 3.7671 km
	A with distance 4.1564 km
NAME C_alt closest matches:
C_alt [0.         0.00059128 0.00063334] [1 0 6]
	C with distance 0.0000 km
	B with distance 3.7671 km
	H with distance 4.0350 km
